# Stripe comments form python codes 

In [21]:
import re

def remove_comments(code):
    # Remove single line comments
    code = re.sub(r'#.*$', '', code, flags=re.MULTILINE)

    # Remove multiline comments
    code = re.sub(r"'''[\s\S]*?'''", '', code)
    code = re.sub(r'"""[\s\S]*?"""', '', code)
    code = re.sub(r'\n\s*\n', '\n\n', code)
    return code

def read_code_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

if __name__ == "__main__":
    # Replace 'your_file_path.py' with the path to the file you want to read
    file_path = 'your_source_code.py'

    code = read_code_file(file_path)
    cleaned_code = remove_comments(code)
    with open(file_path+'_withoutComments.py', 'w') as file:
        file.write(cleaned_code)


In [ ]:
def reverse_diffusion(model, sd, timesteps=1000, vector_dim=128, 
                      num_vectors=5, nrow=8, device="cpu", **kwargs):
 
    x = torch.randn((num_vectors, vector_dim), device=device)
    model.eval()
 
    for time_step in tqdm(iterable=reversed(range(1, timesteps)), 
                          total=timesteps-1, dynamic_ncols=False, 
                          desc="Sampling :: ", position=0):
 
        ts = torch.ones(num_vectors, dtype=torch.long, device=device) * time_step
        z = torch.randn_like(x) if time_step > 1 else torch.zeros_like(x)
 
        predicted_noise = model(x, ts)
 
        beta_t                            = sd.beta[ts].unsqueeze(-1)
        one_by_sqrt_alpha_t               = sd.one_by_sqrt_alpha[ts].unsqueeze(-1)
        sqrt_one_minus_alpha_cumulative_t = sd.sqrt_one_minus_alpha_cumulative[ts].unsqueeze(-1)
 
        x = (
            one_by_sqrt_alpha_t
            * (x - (beta_t / sqrt_one_minus_alpha_cumulative_t) * predicted_noise)
            + torch.sqrt(beta_t) * z
        )
    print(x)
    return None

# pytroch test

In [ ]:


from typing import Optional, List

import numpy as np
import torch

class DDIMSampler():
    def __init__(self, model, forward_diffusion, n_steps: int, ddim_discretize: str = "uniform", ddim_eta: float = 0.):

        super().__init__(model)

        self.n_steps = forward_diffusion.timesteps

        if ddim_discretize == 'uniform':
            c = self.n_steps // n_steps
            self.time_steps = np.asarray(list(range(0, self.n_steps, c))) + 1

        elif ddim_discretize == 'quad':
            self.time_steps = ((np.linspace(0, np.sqrt(self.n_steps * .8), n_steps)) ** 2).astype(int) + 1
        else:
            raise NotImplementedError(ddim_discretize)

        with torch.no_grad():

            alpha_bar = self.forward_diffusion.alpha_cumulative

            self.ddim_alpha = alpha_bar[self.time_steps].clone().to(torch.float32)

            self.ddim_alpha_sqrt = torch.sqrt(self.ddim_alpha)

            self.ddim_alpha_prev = torch.cat([alpha_bar[0:1], alpha_bar[self.time_steps[:-1]]])

            self.ddim_sigma = (ddim_eta *
                               ((1 - self.ddim_alpha_prev) / (1 - self.ddim_alpha) *
                                (1 - self.ddim_alpha / self.ddim_alpha_prev)) ** .5)

            self.ddim_sqrt_one_minus_alpha = (1. - self.ddim_alpha) ** .5

    @torch.no_grad()
    def sample(self,
               shape: List[int],
               repeat_noise: bool = False,
               temperature: float = 1.,
               x_last: Optional[torch.Tensor] = None,
               uncond_scale: float = 1.,
               uncond_cond: Optional[torch.Tensor] = None,
               skip_steps: int = 0,
               ):

        device = self.forward_diffusion.device
        bs = shape[0]

        x = x_last if x_last is not None else torch.randn(shape, device=device)

        time_steps = np.flip(self.time_steps)[skip_steps:]

        for i, step in enumerate(time_steps):

            index = len(time_steps) - i - 1

            ts = x.new_full((bs,), step, dtype=torch.long)

            x, pred_x0, e_t = self.p_sample(x, ts, step, index=index,
                                            repeat_noise=repeat_noise,
                                            temperature=temperature,
                                            uncond_scale=uncond_scale,
                                            uncond_cond=uncond_cond)

        return x

    @torch.no_grad()
    def p_sample(self, x: torch.Tensor, t: torch.Tensor, step: int, index: int, *,
                 repeat_noise: bool = False,
                 temperature: float = 1.,
                 uncond_scale: float = 1.,
                 uncond_cond: Optional[torch.Tensor] = None):

        e_t = self.model(x,t)

        x_prev, pred_x0 = self.get_x_prev_and_pred_x0(e_t, index, x,
                                                      temperature=temperature,
                                                      repeat_noise=repeat_noise)

        return x_prev, pred_x0, e_t

    def get_x_prev_and_pred_x0(self, e_t: torch.Tensor, index: int, x: torch.Tensor, *,
                               temperature: float,
                               repeat_noise: bool):

        alpha = self.ddim_alpha[index]

        alpha_prev = self.ddim_alpha_prev[index]

        sigma = self.ddim_sigma[index]

        sqrt_one_minus_alpha = self.ddim_sqrt_one_minus_alpha[index]

        pred_x0 = (x - sqrt_one_minus_alpha * e_t) / (alpha ** 0.5)

        dir_xt = (1. - alpha_prev - sigma ** 2).sqrt() * e_t

        if sigma == 0.:
            noise = 0.

        elif repeat_noise:
            noise = torch.randn((1, *x.shape[1:]), device=x.device)

        else:
            noise = torch.randn(x.shape, device=x.device)

        noise = noise * temperature

        x_prev = (alpha_prev ** 0.5) * pred_x0 + dir_xt + sigma * noise

        return x_prev, pred_x0